In [1]:
#load libraries
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from itertools import cycle, islice
import weightedcalcs as wc

# This line lets us plot on our ipython notebook
%matplotlib inline

In [2]:
pov = pd.read_table("proj_data/pov_guidelines.csv", sep=',', low_memory=False)

pov15=pov.iloc[:,[0,1]]
pov10=pov.iloc[:,[0,6]]
pov05=pov.iloc[:,[0,11]]


In [3]:
def filtercolumns(pumsdata, povdata):
    """
    function filtercolumns pulls relevant columns from pums households data (csv)
    and joins with poverty data based on hhd_size, povdata 
    (a df of just that year's poverty guidelines) 
    returns df of pums data with joined pov data
    """
    pums = pd.read_table(pumsdata,sep=",", low_memory=False)
    df_relevant = pums[['SERIALNO','ST', 'PUMA', 'ADJHSG', 'WGTP', 'NP', 'TEN', 'BLD', 'BDSP', 'RNTP', 'VACS','HINCP','SMOCP','GRNTP', 'GRPIP','OCPIP', 'KIT', 'PLM']]
    pums_pov = df_relevant.merge(povdata, left_on='NP', right_on='hhd_size')
    return pums_pov

# test code
# pums15 = "proj_data/ss15hpa_1yr.csv"
# fd15=filtercolumns(pums15,pov15)



In [4]:
def calcRpov(data, yy, exportname=""):
    """
    calculates % of renting HH below federal poverty line in each 
    PUMA. 
    requires three arguments, 
    data is pums df with joined pov data
    yy is 2-digit year, eg 15 for 2015
    exportname is export csv name
    returns pct poverty by puma as DF and CSV
    
    """
    povyear = "pov" + "_" + str(yy)
    r = data[data['TEN']==3]
#     print("r_ten:", len(r.index))
    rK = r[r['KIT']==1]
    rKP = rK[rK['PLM']==1]
#     print("r_kit,plm", len(rKP.index))
    rKP['WGTPpov'] = rKP['WGTP'].where(rKP['HINCP']<=rKP[povyear], 0)
    #group by Puma
    grp = rKP.groupby(["PUMA"])
    povsum = grp[['WGTP','WGTPpov']].sum()
    povsum['PCTpov'] = (povsum['WGTPpov'] / povsum['WGTP'])*100
#     rNeed = rKP[(rKP['HINCP']<=rKP[povyear]) & (rKP['GRPIP'].notnull())]
#     print("rbelow", len(rNeed.index))
    if len(exportname)>0:
        povsum.to_csv(exportname)
    else:
        return povsum




In [6]:
def calcBurden(data,povyear):
    povyear = "pov" + "_" + str(yy)
    r = data[data['TEN']==3]
#     print("r_ten:", len(r.index))
    rK = r[r['KIT']==1]
    rKP = rK[rK['PLM']==1]
    
    # FILTER for records where hincp is less than poverty guideline
    need = rKP[(rKP['HINCP']<=rKP[povyear]) & (rKP['GRPIP'].notnull())]
    need['WGTP30'] = need['WGTP'].where(need['GRPIP']>=30.0, 0)
    return need

pums10="proj_data/ss10hpa.csv"

calcRpov(filtercolumns(pums10, pov10), 10)

# calcBurden(calcRpov(filtercolumns(pums10, pov10), 10))

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


WGTP  WGTPpov     PCTpov
PUMA                           
100   19435     8528  43.879599
200   13939     3174  22.770644
300   12698     4195  33.036699
400    7770     2063  26.550837
500    7961     2389  30.008793
600   10358     2843  27.447384
700    9290     2525  27.179763
801   18307     3748  20.473043
802    9587     1751  18.264316
901   16799     6505  38.722543
902    9362     2370  25.315104
903   16856     3786  22.460845
1000  13405     4166  31.077956
1100  11276     1982  17.577155
1200  15920     3807  23.913317
1300  21814     7091  32.506647
1400   9989     3929  39.333267
1500   7514     2975  39.592760
1600  10454     2757  26.372680
1701  20796     7038  33.843047
1702  22933     7069  30.824576
1703  17471     4964  28.412798
1801  12471     2345  18.803624
1802  13787     2825  20.490317
1803  14331     1443  10.069081
1804  24772     5924  23.914097
1805  14356     1581  11.012817
1806   7764     1090  14.039155
1807  15796     5309  33.609775
1900  15154     3111  20.529233
...     ...      ...        ...
3801  15523     5242  33.769246
3802   9909      995  10.041377
3901  13034     1178   9.037901
3902   7177     1286  17.918350
3903   9643      876   9.084310
3904  14288     1482  10.372340
4001   8925     1293  14.487395
4002  14445     1463  10.128072
4003  13457     1284   9.541503
4004  17067     2683  15.720396
4005  11269     1888  16.753927
4006  12102     1052   8.692778
4101  17409     3227  18.536389
4102  16023     4959  30.949260
4103  16333     6938  42.478418
4104  20376     6262  30.732234
4105  23141    11719  50.641718
4106  21845     5100  23.346304
4107  22800    12106  53.096491
4108  23005    11387  49.497935
4109  38056     5360  14.084507
4110  16109     4391  27.258055
4111  27707    11661  42.086837
4201   9221      783   8.491487
4202  17968     3304  18.388246
4203  17512     4146  23.675194
4204   9677      365   3.771830
4301  12419      404   3.253080
4302  15948     2166  13.581640
4303  12742     2277  17.870036

[92 rows x 3 columns]

In [ ]:

calcRpov(filtercolumns(pums10, pov10), 10,'r10_poverty.csv')